In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import tqdm
import xarray as xr
from IPython.display import HTML
from matplotlib import animation

In [ ]:
da_low = xr.open_dataarray("../data/01_raw/nextgems/cf_global_10km.nc")
da_high = xr.open_dataarray("../data/01_raw/nextgems/ws_global_10km.nc")
da_low

In [ ]:
da_low.isel(time=10).fillna(0)

In [ ]:
def animate(da: xr.DataArray) -> animation:
    fig, ax = plt.subplots(1, 1, figsize=[12, 6])
    vmin = float(da.min())
    vmax = float(da.max())

    def plot_step(time):
        """Plot a time step of the animation."""
        da.isel(time=time).plot(ax=ax, add_colorbar=False, vmin=vmin, vmax=vmax)

    return animation.FuncAnimation(fig, plot_step, 168, interval=50, blit=False)

In [ ]:
ani = animate(da_low)

In [ ]:
writer = animation.FFMpegWriter(fps=20)
ani.save("sst_low.mp4", writer=writer)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Orthographic(-10, 45))
da_low.sel(time="2040-09-01 00:00:00").plot.imshow(ax=ax, transform=ccrs.PlateCarree())

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Orthographic(-10, 45))
da_high.sel(time="2040-09-01 00:00:00").plot.imshow(ax=ax, transform=ccrs.PlateCarree())

In [ ]:
fig = plt.figure(figsize=(12, 12))


def plot_step(frame, fig):
    """Plot a time step of the animation."""
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Orthographic(0 + frame, 30))
    da_low.isel(time=frame % 120).plot.imshow(
        add_colorbar=False, ax=ax, cmap="viridis", transform=ccrs.PlateCarree()
    )
    ax.set_title("")


ani = animation.FuncAnimation(
    fig, plot_step, 360, interval=50, blit=False, fargs=(fig,)
)

In [ ]:
HTML(ani.to_html5_video())

In [ ]:
ani.save("globe_ws_low_res.mp4")

In [ ]:
def animate(da: xr.DataArray) -> animation.ArtistAnimation:
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Orthographic(0, 30))

    vmin = float(da.min())
    vmax = float(da.max())

    ims = []

    for i in tqdm(range(168)):
        im = da.isel(time=i).plot.imshow(
            add_colorbar=False,
            ax=ax,
            cmap="viridis",
            transform=ccrs.PlateCarree(),
            animated=True,
            vmin=vmin,
            vmax=vmax,
        )
        ax.set_title("")
        ims.append([im])

    return animation.ArtistAnimation(
        fig, ims, interval=50, blit=True, repeat_delay=1000
    )

In [ ]:
ani = animate(da_low)

In [ ]:
writer = animation.FFMpegWriter(fps=20)
ani.save("globe_ws_low_res_rot.mp4", writer=writer)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Orthographic(0, 30))

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Orthographic(0, 30))

vmin = float(da_low.min())
vmax = float(da_low.max())
mesh = da_low.isel(time=0).plot(
    add_colorbar=False,
    ax=ax,
    cmap="viridis",
    transform=ccrs.PlateCarree(),
    vmin=vmin,
    vmax=vmax,
)


def update_mesh(t, fig):
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Orthographic(t, 30))  # noqa: F841
    mesh.set_array(da_low.isel(time=t % 168))


anim = animation.FuncAnimation